# Parallel computing on multivectors

In [1]:
using Distributed
using DistributedArrays
addprocs(4);

In [2]:
@everywhere using JOLI
@everywhere using LinearAlgebra

#### Setup problem size, define operator, and input

In [3]:
M=8;
NVC=15;

In [4]:
A=joDFT(M,planned=false);
show(A)

Type: joLinearFunction
Name: joDFT
Size: (8, 8)
 DDT: Float64
 RDT: Complex{Float64}


In [5]:
x=randn(M,NVC)

8×15 Array{Float64,2}:
 -0.270754  -1.3369     -1.06987    …  -0.558707  -1.60898    -0.0924695
 -0.60623   -1.67037     0.0420038      0.342735   0.277882   -0.250557 
  0.98971   -0.0660405  -2.57101       -0.176547  -0.427845    0.549751 
  0.372959   1.26178     0.153234      -1.1445    -1.51067     0.0844326
 -1.19674   -0.136317   -0.571931       0.110122   0.111624    0.662711 
 -1.01415   -2.42453    -1.363      …   2.26607    0.153304    0.0213506
  1.53771    0.673964    0.95488        0.823318  -0.127601    0.773563 
 -0.251367   0.715949   -0.0310003     -1.55734   -0.0625879   1.02815  

## Using Either Ararys

#### Query/Set parallel mode

In [6]:
pmode=jo_PAmode(verbose=true)

Parallel mode is set to :SA (shared arrays)


:SA

In [7]:
jo_PAmode(:SA) # :SA for shared or :DA for distributed
pmode=jo_PAmode()

:SA

#### Define parallel setup

In [8]:
ps=joPAsetup((M,NVC));
show(ps)

joPAsetup: joPAsetup
 DataType: Float64
 Dims    : (8, 15)
 Chunks  : [1, 4]
 Workers : [2, 3, 4, 5]
  Worker/ranges:   2 (1:8, 1:4)
  Worker/ranges:   3 (1:8, 5:8)
  Worker/ranges:   4 (1:8, 9:12)
  Worker/ranges:   5 (1:8, 13:15)


#### Define parallel operator

In [9]:
pA=joPAdistributedLinOp(A,ps);
show(pA)

Type: joSAdistributedLinearOperator
Name: joSAdistributedLinearOperator(joDFT)
Size: (8, 8)
 NVC: 15
 DDT: Float64
 RDT: Complex{Float64}


#### Define shared/distributed array

In [10]:
@everywhere function fillin(x,n)
    println(myid(),n,(length(n[1]),length(n[2])))
    x[n...]
end

if pmode==:SA
    px=salloc(ps)
    px[:,:]=x[:,:]
elseif pmode==:DA
    px=dfill(r->fillin(x,r),ps)
end
px

8×15 SharedArrays.SharedArray{Float64,2}:
 -0.270754  -1.3369     -1.06987    …  -0.558707  -1.60898    -0.0924695
 -0.60623   -1.67037     0.0420038      0.342735   0.277882   -0.250557 
  0.98971   -0.0660405  -2.57101       -0.176547  -0.427845    0.549751 
  0.372959   1.26178     0.153234      -1.1445    -1.51067     0.0844326
 -1.19674   -0.136317   -0.571931       0.110122   0.111624    0.662711 
 -1.01415   -2.42453    -1.363      …   2.26607    0.153304    0.0213506
  1.53771    0.673964    0.95488        0.823318  -0.127601    0.773563 
 -0.251367   0.715949   -0.0310003     -1.55734   -0.0625879   1.02815  

### Just parallel

In [11]:
py=pA*px

8×15 SharedArrays.SharedArray{Complex{Float64},2}:
 -0.155163+0.0im         -1.05446+0.0im        …   0.981796+0.0im     
  0.273285-0.0643151im  -0.372388-0.0633667im     -0.099043+0.383037im
  -1.41242+0.615882im   -0.735795+2.147im         -0.266251+0.474396im
  0.381486-0.45181im    -0.476554-0.586629im       -0.43495+0.224778im
  0.904644+0.0im         0.442604+0.0im            0.357151+0.0im     
  0.381486+0.45181im    -0.476554+0.586629im   …   -0.43495-0.224778im
  -1.41242-0.615882im   -0.735795-2.147im         -0.266251-0.474396im
  0.273285+0.0643151im  -0.372388+0.0633667im     -0.099043-0.383037im

### Serial to serial via parallel

#### Define distrtibuting and gathering operators

In [12]:
pD=joPAdistribute(pA);
show(pD)

Type: joSAdistribute
Name: joSAdistributeMV:15
Size: (8, 8)
 NVC: 15
 DDT: Float64
 RDT: Float64


In [13]:
pG=joPAgather(pA);
show(pG)

Type: joSAgather
Name: joSAgatherMV:15
Size: (8, 8)
 NVC: 15
 DDT: Complex{Float64}
 RDT: Complex{Float64}


#### Distribute data, apply operator, gather data

In [14]:
px=pD*x

8×15 SharedArrays.SharedArray{Float64,2}:
 -0.270754  -1.3369     -1.06987    …  -0.558707  -1.60898    -0.0924695
 -0.60623   -1.67037     0.0420038      0.342735   0.277882   -0.250557 
  0.98971   -0.0660405  -2.57101       -0.176547  -0.427845    0.549751 
  0.372959   1.26178     0.153234      -1.1445    -1.51067     0.0844326
 -1.19674   -0.136317   -0.571931       0.110122   0.111624    0.662711 
 -1.01415   -2.42453    -1.363      …   2.26607    0.153304    0.0213506
  1.53771    0.673964    0.95488        0.823318  -0.127601    0.773563 
 -0.251367   0.715949   -0.0310003     -1.55734   -0.0625879   1.02815  

In [15]:
py=pA*px

8×15 SharedArrays.SharedArray{Complex{Float64},2}:
 -0.155163+0.0im         -1.05446+0.0im        …   0.981796+0.0im     
  0.273285-0.0643151im  -0.372388-0.0633667im     -0.099043+0.383037im
  -1.41242+0.615882im   -0.735795+2.147im         -0.266251+0.474396im
  0.381486-0.45181im    -0.476554-0.586629im       -0.43495+0.224778im
  0.904644+0.0im         0.442604+0.0im            0.357151+0.0im     
  0.381486+0.45181im    -0.476554+0.586629im   …   -0.43495-0.224778im
  -1.41242-0.615882im   -0.735795-2.147im         -0.266251-0.474396im
  0.273285+0.0643151im  -0.372388+0.0633667im     -0.099043-0.383037im

In [16]:
y=pG*py

8×15 Array{Complex{Float64},2}:
 -0.155163+0.0im         -1.05446+0.0im        …   0.981796+0.0im     
  0.273285-0.0643151im  -0.372388-0.0633667im     -0.099043+0.383037im
  -1.41242+0.615882im   -0.735795+2.147im         -0.266251+0.474396im
  0.381486-0.45181im    -0.476554-0.586629im       -0.43495+0.224778im
  0.904644+0.0im         0.442604+0.0im            0.357151+0.0im     
  0.381486+0.45181im    -0.476554+0.586629im   …   -0.43495-0.224778im
  -1.41242-0.615882im   -0.735795-2.147im         -0.266251-0.474396im
  0.273285+0.0643151im  -0.372388+0.0633667im     -0.099043-0.383037im

#### Or all together

In [17]:
ps2s=pG*pA*pD
show(ps2s)

Type: joLinearOperator
Name: ((joSAgatherMV:15*joSAdistributedLinearOperator(joDFT))*joSAdistributeMV:15)
Size: (8, 8)
 DDT: Float64
 RDT: Complex{Float64}


In [18]:
y=ps2s*x

8×15 Array{Complex{Float64},2}:
 -0.155163+0.0im         -1.05446+0.0im        …   0.981796+0.0im     
  0.273285-0.0643151im  -0.372388-0.0633667im     -0.099043+0.383037im
  -1.41242+0.615882im   -0.735795+2.147im         -0.266251+0.474396im
  0.381486-0.45181im    -0.476554-0.586629im       -0.43495+0.224778im
  0.904644+0.0im         0.442604+0.0im            0.357151+0.0im     
  0.381486+0.45181im    -0.476554+0.586629im   …   -0.43495-0.224778im
  -1.41242-0.615882im   -0.735795-2.147im         -0.266251-0.474396im
  0.273285+0.0643151im  -0.372388+0.0633667im     -0.099043-0.383037im